#### Since the data is highly imbalanced, we take 11,000 samples out of each category.
#### The categories being: 
##### 1) Baby/Children
##### 2) Divided
##### 3) Ladieswear
##### 4) Menswear
##### 5) Sport

##### We also remove samples which don't have detail_desc

##### This data will be used to train the image and text embeddings

In [1]:
import os
import pandas as pd
from PIL import Image

In [2]:
articles_df = pd.read_csv("C:/Users/ojubh\/Desktop/SEMESTER 2/Deep Learning/Final_Project/data/articles.csv")

In [3]:
len(articles_df)

105542

In [4]:
articles_df = articles_df[["article_id", "index_group_name", "detail_desc"]]

In [5]:
articles_df.head()

,article_id,index_group_name,detail_desc
0,108775015,Ladieswear,Jersey top with narrow shoulder straps.
1,108775044,Ladieswear,Jersey top with narrow shoulder straps.
2,108775051,Ladieswear,Jersey top with narrow shoulder straps.
3,110065001,Ladieswear,"Microfibre T-shirt bra with underwired, moulde..."
4,110065002,Ladieswear,"Microfibre T-shirt bra with underwired, moulde..."


In [6]:
articles_df["detail_desc"].isna().unique()

array([False,  True])

In [7]:
articles_df.dropna(subset=['detail_desc'], how='all', inplace=True)

In [8]:
## missing values have been removed
articles_df["detail_desc"].isna().unique()

array([False])

In [9]:
articles_df["index_group_name"].unique()

array(['Ladieswear', 'Baby/Children', 'Menswear', 'Sport', 'Divided'],
      dtype=object)

In [10]:
index_group = articles_df.groupby("index_group_name")

In [11]:
for name, group in index_group:
    print('name:', name, "|", "size:", len(group), "%:", len(group)/len(articles_df))

name: Baby/Children | size: 34619 %: 0.3293095903962864
name: Divided | size: 15086 %: 0.14350398569335845
name: Ladieswear | size: 39523 %: 0.3759583737610106
name: Menswear | size: 12539 %: 0.11927591651922455
name: Sport | size: 3359 %: 0.03195213363012005


In [12]:
training_split = int(0.6*len(articles_df))
print("Training split", training_split)
print("Samples for Baby/Child:", int(0.32*training_split))
print("Samples for Divided:", int(0.14*training_split))
print("Samples for Ladieswear:", int(0.37*training_split))
print("Samples for menswear", int(0.11 * training_split))
print("samples for sport", int(0.03 * training_split))


Training split 63075
Samples for Baby/Child: 20184
Samples for Divided: 8830
Samples for Ladieswear: 23337
Samples for menswear 6938
samples for sport 1892


In [13]:
#no sampling for "Sport"
sport_df_train = articles_df[articles_df["index_group_name"] == "Sport"].sample(n=int(0.03 * training_split))

baby_children_df_train = articles_df[articles_df["index_group_name"] == "Baby/Children"].sample(n=int(0.32*training_split))

divided_df_train = articles_df[articles_df["index_group_name"] == "Divided"].sample(n=int(0.14*training_split))

ladieswear_df_train = articles_df[articles_df["index_group_name"] == "Ladieswear"].sample(n=int(0.37*training_split))

menswear_df_train = articles_df[articles_df["index_group_name"] == "Menswear"].sample(n=int(0.11 * training_split))




train_df = pd.concat([sport_df_train,
                        baby_children_df_train,
                        divided_df_train,
                        ladieswear_df_train,
                        menswear_df_train], axis=0)

In [14]:
train_df.head(10)

,article_id,index_group_name,detail_desc
84723,816752002,Sport,Backpack in nylon with a handle at the top and...
23528,605777009,Sport,"Hooded sports top in airy, fast-drying jersey ..."
24403,609170001,Sport,Sports trousers in fast-drying functional fabr...
75897,783335001,Sport,Running top in fast-drying functional fabric w...
44453,680034001,Sport,Fine-knit shaftless yoga socks with elasticate...
14654,564044001,Sport,Wide sports top in fast-drying functional fabr...
1613,362997012,Sport,Sports vest top in fast-drying functional fabr...
28655,623781010,Sport,Short-sleeved sports tops in fast-drying funct...
92461,853881001,Sport,Sports shorts in fast-drying functional fabric...
98691,880439001,Sport,Basketball-inspired sports vest top in fast-dr...


In [15]:
articles_df = articles_df.drop(train_df.index)

In [19]:
len(articles_df)

43945

In [16]:
train_df.reset_index()

,index,article_id,index_group_name,detail_desc
0,84723,816752002,Sport,Backpack in nylon with a handle at the top and...
1,23528,605777009,Sport,"Hooded sports top in airy, fast-drying jersey ..."
2,24403,609170001,Sport,Sports trousers in fast-drying functional fabr...
3,75897,783335001,Sport,Running top in fast-drying functional fabric w...
4,44453,680034001,Sport,Fine-knit shaftless yoga socks with elasticate...
...,...,...,...,...
61176,12331,552743029,Menswear,"Long-sleeved jumper in soft, fine-knit cashmer..."
61177,68538,754429001,Menswear,Twill cap with an adjustable tab and metal fas...
61178,32144,634013012,Menswear,"Oxford cotton shirt with a button-down collar,..."
61179,82478,808714002,Menswear,Oversized top in sweatshirt fabric with a line...


In [20]:
val_split = int(len(articles_df)/2)

print("Val split", val_split)
print("Samples for Baby/Child:", int(0.32*val_split))
print("Samples for Divided:", int(0.14*val_split))
print("Samples for Ladieswear:", int(0.37*val_split))
print("Samples for menswear", int(0.11 * val_split))
print("samples for sport", int(0.03 * val_split))

Val split 21972
Samples for Baby/Child: 7031
Samples for Divided: 3076
Samples for Ladieswear: 8129
Samples for menswear 2416
samples for sport 659


In [21]:
sport_df_val = articles_df[articles_df["index_group_name"] == "Sport"].sample(n=int(0.03 * val_split))

baby_children_df_val = articles_df[articles_df["index_group_name"] == "Baby/Children"].sample(n=int(0.32*val_split))

divided_df_val = articles_df[articles_df["index_group_name"] == "Divided"].sample(n=int(0.14*val_split))

ladieswear_df_val = articles_df[articles_df["index_group_name"] == "Ladieswear"].sample(n=int(0.37*val_split))

menswear_df_val = articles_df[articles_df["index_group_name"] == "Menswear"].sample(n=int(0.11 * val_split))


val_df = pd.concat([sport_df_val,
                        baby_children_df_val,
                        divided_df_val,
                        ladieswear_df_val,
                        menswear_df_val], axis=0)

In [22]:
test_df = articles_df.drop(val_df.index)

In [23]:
val_df.reset_index()
test_df.reset_index()

,index,article_id,index_group_name,detail_desc
0,2,108775051,Ladieswear,Jersey top with narrow shoulder straps.
1,9,111593001,Ladieswear,"Semi shiny tights that shape the tummy, thighs..."
2,12,112679052,Baby/Children,Sweatshirt in soft organic cotton with a pres...
3,21,118458038,Menswear,Trousers in sweatshirt fabric with an elastica...
4,23,120129001,Ladieswear,Leggings in soft jersey with a wide panel at t...
...,...,...,...,...
22629,105523,948152002,Baby/Children,Set with a long-sleeved top and a skirt in sof...
22630,105528,949198001,Ladieswear,Joggers in sweatshirt fabric made from a cotto...
22631,105532,949594001,Ladieswear,Joggers in soft sweatshirt fabric with an elas...
22632,105536,952938001,Ladieswear,Fitted top in jersey with a round neckline and...


In [25]:
len(val_df) + len(test_df) + len(train_df)

105126

In [26]:
def get_image_path(balanced_df):
    final_rows = []
    mypath = os.path.join('c:', os.sep, 'Users', "ojubh", "Desktop", "images")

    for (_, row) in balanced_df.iterrows():
        location = "0" + str(row["article_id"])[:2] + "/" + "0" + str(row["article_id"]) + ".jpg"
        path = os.path.join(mypath, *location.split("/"))
        if os.path.isfile(path):
            final_rows.append([row["article_id"], row["index_group_name"], row["detail_desc"], location])
    
    return final_rows

In [27]:
train_rows = get_image_path(train_df)
val_rows = get_image_path(val_df)
test_rows = get_image_path(test_df)

# final_df = pd.DataFrame(final_rows, columns = ["article_id", "index_group_name", "detail_desc", "location"] )

In [28]:
len(train_rows) + len(val_rows) + len(test_rows)

104696

In [29]:
train_final = pd.DataFrame(train_rows, columns = ["article_id", "index_group_name", "detail_desc", "location"])
val_final = pd.DataFrame(val_rows, columns = ["article_id", "index_group_name", "detail_desc", "location"] )
test_final = pd.DataFrame(test_rows, columns = ["article_id", "index_group_name", "detail_desc", "location"] )

In [31]:

import shutil

def write_images(final_df, tag=""):
    image_paths = final_df["location"].tolist()
    dest_folder = tag + "_images"
    dest_path = os.path.join('c:', os.sep, 'Users', "ojubh", "Desktop", dest_folder)
    os.mkdir(dest_path)

    mypath = os.path.join('c:', os.sep, 'Users', "ojubh", "Desktop", "images")
    newsize = (256, 256)
    for image in image_paths:
        img_path = os.path.join(mypath, *image.split("/"))
        img = Image.open(img_path)
        img = img.resize(newsize)
        dest_loc = os.path.join(dest_path, image.split("/")[1])
        img.save(dest_loc)


In [32]:
write_images(train_final, "train")

In [33]:
write_imagesages(val_final, "val")

In [34]:
write_images(test_final, "test")

In [35]:
# export dataframe to csv
train_final.to_csv("train_data.csv")
val_final.to_csv("val_data.csv")
test_final.csv("test_data.csv")

AttributeError: 'DataFrame' object has no attribute 'csv'